Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=49762ea8addc770377430acfe666ca217bd67f5841d7b05b85fac3a2812efad2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [26]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
import pandas as pd

In [34]:
# Create a Spark session
spark = SparkSession.builder.appName("salary_analysis").getOrCreate()

# Create dataframe
salary_df = spark.createDataFrame([
  (1, "Joe", 70, 3),
  (2, "Henry", 80, 4),
  (3, "Sam", 60, "Null"),
  (4, "Max", 90, "Null"),
], ["ID", "Name", "Salary", "Manager_ID"])
salary_df.show()

# Create additional table with managers'salary only
managers_df = salary_df.select("ID", "Salary").where(salary_df.Manager_ID == "Null").withColumnRenamed("ID", "Manager_id").withColumnRenamed("Salary", "Manager_salary")
managers_df.show()

# Сombining two tables
join_df = salary_df.join(managers_df, salary_df["Manager_ID"] == managers_df["Manager_id"], "inner")
join_df.show()

#Result
join_df.filter(join_df['Salary'] > join_df['Manager_salary']).select(join_df.Name).show()

+---+-----+------+----------+
| ID| Name|Salary|Manager_ID|
+---+-----+------+----------+
|  1|  Joe|    70|         3|
|  2|Henry|    80|         4|
|  3|  Sam|    60|      Null|
|  4|  Max|    90|      Null|
+---+-----+------+----------+

+----------+--------------+
|Manager_id|Manager_salary|
+----------+--------------+
|         3|            60|
|         4|            90|
+----------+--------------+

+---+-----+------+----------+----------+--------------+
| ID| Name|Salary|Manager_ID|Manager_id|Manager_salary|
+---+-----+------+----------+----------+--------------+
|  1|  Joe|    70|         3|         3|            60|
|  2|Henry|    80|         4|         4|            90|
+---+-----+------+----------+----------+--------------+

+----+
|Name|
+----+
| Joe|
+----+

